# How to register and track your model with Layer

[![Open in Layer](https://app.layer.ai/assets/badge.svg)](https://app.layer.ai/layer/titanic) [![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/layerai/examples/blob/main/titanic/Getting_Started_With_Layer.ipynb) [![Layer Examples Github](https://badgen.net/badge/icon/github?icon=github&label)](https://github.com/layerai/examples/tree/main/titanic)

Layer helps you build, train and track all your machine learning project metadata including ML models and datasets‍ with semantic versioning, extensive artifact logging and dynamic reporting with local↔cloud training.

In this quick walkthrough, we'll take a look at how to register and track ML models with Layer.

## Installation

Let's start with installing [Layer](https://layer.ai).

In [ ]:
!pip install -U layer -q

In [ ]:
from sklearn.datasets import fetch_california_housing
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from layer.decorators import model
import seaborn as sns
import xgboost as xgb
import pandas as pd
import layer

## Inititialize Your First Layer Project

It's time to create your first [Layer Project](https://docs.app.layer.ai/docs/projects/layer-projects). 

Layer projects are basically containers to organize your machine learning project assets. They associate some combination of datasets and models. Layer projects are basically the front page of your ML projects which includes all your ml metadata including ML models, datasets, metric, parameters and more.

In [58]:
# Log in to your free account!
layer.login()

In [59]:
layer.init("house_price_prediction")

Project(name='house_price_prediction', raw_datasets=[], derived_datasets=[], models=[], path=PosixPath('.'), project_files_hash='', readme='', account=Account(id=UUID('add1b570-c8e7-4187-b747-1d01104893a9'), name='layer'), _id=UUID('7c7372bd-96b5-4182-b53a-cae1f3b7df78'), functions=[])

⬆️ Click this link to visit your Layer Project page.

## Train and Register Your Model

In [60]:
X, y = fetch_california_housing(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In this tutorial, we will be training a `XGBRegressor` to predict the house prices. We can see that this is a simple training function which returns the trained model and prints the `mean_square_error` of our model.

In [61]:
def train():
    xgb_model = xgb.XGBRegressor()
    xgb_model.fit(X_train, y_train)    
    predictions = xgb_model.predict(X_test)

    mse = mean_squared_error(y_test, predictions)    
    print("mse: ", mse)
    
    return xgb_model

xgb_model = train()

mse:  0.2217551006863001


### 🔥 Now let's register our model to Layer!

Simply add the `@model` decorator to our training function. Layer will register the returning trained model to your Layer Project. Let's also replace `print()` with `layer.log()` to enable experiment tracking.

In [62]:
@model("california_housing")
def train():
    xgb_model = xgb.XGBRegressor()
    xgb_model.fit(X_train, y_train)    
    predictions = xgb_model.predict(X_test)

    mse = mean_squared_error(y_test, predictions)    
    layer.log({"mse": mse})
    
    return xgb_model

xgb_model = train()

Output()

⬆️ Click the above link to see the registered model in your Layer Project. You will see that Layer automatically registered and versioned your model.

Let's do some manual hyperparameter tuning and compare our experiments if it changed anything on the `mse` metric.

In [63]:
@model("california_housing")
def train():
    xgb_model = xgb.XGBRegressor(max_depth=parameters["max_depth"], n_estimators=parameters["n_estimators"])
    xgb_model.fit(X_train, y_train)
    predictions = xgb_model.predict(X_test)
    
    mse = mean_squared_error(y_test, predictions)    
    layer.log(parameters)
    layer.log({"mse": mse})
    
    return xgb_model

parameters = {"max_depth":1, "n_estimators":10}
xgb_model = train()

Output()

⬆️ Click the above link, you will find the new version of your model with the `max_depth` and `n_estimators` parameters.

![Layer compare experiments](https://raw.githubusercontent.com/layerai/examples/main/tutorials/add-models-to-layer/assets/layer_compare_experiments.png)

You can log plots and tables with `layer.log` too. Let's log some more data to our house price prediction experiment.

In [ ]:
@model("california_housing")
def train():
    xgb_model = xgb.XGBRegressor(max_depth=parameters["max_depth"], n_estimators=parameters["n_estimators"])
    xgb_model.fit(X_train, y_train)    
    predictions = xgb_model.predict(X_test)
    
    mse = mean_squared_error(y_test, predictions)    
    table = pd.DataFrame(zip(predictions,y_test),columns=['Predicted Price', "Actual Price"])
    plot = sns.histplot((y_test-predictions), kde=True).figure
    layer.log({"Predictions":plot})
    layer.log({"Predictions vs Actual":table[:50]})
    layer.log(parameters)
    layer.log({"mse": mse})
    
    return xgb_model

parameters = {"max_depth":4, "n_estimators":100}
xgb_model = train()

⬆️ Clicking the above link you will find the logged tables and plots.

### 🌐 You can also train your model on Layer infra.

Layer is a very advanced metadata store where you can store your models, datasets but also your pipelines. Just like you registered your model, you can also register and execute your machine learning pipeline on Layer. This is especially usefull because:
- You don't have to deal with the complexities of setting up the remote infrastructure
- 

Now instead of running your `train` function directly, just pass it to Layer with `layer.run([train])`. Layer will pickle your function and run it on Layer infra.

In [54]:
layer.run([train])

Output()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Run(project_name='getting-started')

## 🚀 How to load and use your model from Layer

Once you register your model to Layer, you can load your model from Layer with simple calling `layer.get_model(MODEL_NAME)`

Now, let's see how to load and make predictions with your model:

In [56]:
xgb_model = layer.get_model("california_housing").get_train()

X, y = fetch_california_housing(return_X_y=True)
preds = xgb_model.predict(X)
print("Prediction: ", preds[0])

Output()

Prediction:  4.279534


## Where to go from here?

Now that you have registered your first model to Layer, you can:

- Join our [Slack Community ](https://bit.ly/layercommunityslack) to connect with other Layer users
- Visit [Layer Examples Repo](https://github.com/layerai/examples) for more examples
- Browse [Community Projects](https://layer.ai/community) to see more use cases
- Check out [Layer Documentation](https://docs.layer.ai)
- [Contact us](https://layer.ai/contact-us?interest=notebook) for your questions